## PydanticOutputParser에는 주로 두 가지 핵심 메서드가 구현되어야 합니다.

* get_format_instructions()
    * 이 메서드는 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공합니다.
    * 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 
    * 이 지침은 사용자가 원하는 출력 형식을 정확히 이해하고 준비하는 데 도움을 줍니다.
* parse()
    * 이 메서드는 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환합니다.
    * Pydantic와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다.
    * 이 과정을 통해 언어 모델의 출력을 더 사용하기 쉬운 형태로 가공할 수 있습니다.

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatOpenAI(model="gpt-4o", temperature=0)

email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="보낸 사람의 메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 내용 요약")
    date: str = Field(description="언급된 미팅 날짜")

parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [3]:
# prompt 정의
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

prompt = prompt.partial(format=parser.get_format_instructions())

In [4]:
chain = prompt | llm
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)
print(response.content)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 ZENESIS 자전거 모델의 브로슈어 요청 및 협력 논의를 위한 미팅을 제안함.",
  "date": "1월 15일 오전 10시"
}
```


In [5]:
# 특정 객체로 변환
parser.parse(response.content)

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 ZENESIS 자전거 모델의 브로슈어 요청 및 협력 논의를 위한 미팅을 제안함.', date='1월 15일 오전 10시')

### parser 가 결합된 체인 생성

In [6]:
chain = prompt | llm | parser
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 ZENESIS 자전거 모델의 브로슈어 요청 및 협력 논의를 위한 미팅을 제안함. 미팅 일정은 1월 15일 화요일 오전 10시로 제안됨.', date='1월 15일')